In [1]:
import pandas as pd
import re
import csv
import requests
from bs4 import BeautifulSoup

In [11]:
base_url = "https://sofifa.com/players?offset="
columns = ["ID", "Name", "Age", "Nationality", "Overall Rating", "Potential", "Team", "Value", "Wage Value", "Total Stats"]
data = pd.DataFrame(columns = columns)
print(data)

Empty DataFrame
Columns: [ID, Name, Age, Nationality, Overall Rating, Potential, Team, Value, Wage Value, Total Stats]
Index: []


In [5]:
for offset in range(0, 327):
    url = base_url +str(offset*61)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    tbody = soup.find('tbody')
    for tr in tbody.find_all('tr'):
        td = tr.find_all('td')
        Player_ID = td[0].find('img').get('id')
        name = td[1].find_all('a')[1].text
        age = td[2].text
        Nationality = td[1].find_all('a')[0].get('title')
        Ova =  td[3].text
        Potential = td[4].text
        Team = td[5].find('a').text
        Value = td[6].text
        Wage = td[7].text
        tstats =  td[8].text
        player_data = pd.DataFrame([[Player_ID, name, age, Nationality, Ova, Potential, Team, Value, Wage, tstats]])
        #print(player_data)
        player_data.columns = columns
        data = data.append(player_data, ignore_index = True)
data = data.drop_duplicates()
data.to_csv('basic_data', encoding = "utf-8")

In [3]:
data_csv = pd.read_csv('basic_data')
data = pd.DataFrame(data_csv, columns = columns)

In [4]:
months_correlatives = {"Jan": "1", "Feb": "2", "Mar":"3", "Apr":"4", "May":"5",
                          "Jun": "6", "Jul": "7", "Aug": "8", "Sep": "9", "Oct": "10",
                          "Nov": "11", "Dec": "12"}

In [5]:
print(data.head())

       ID          Name  Age Nationality  Overall Rating  Potential  \
0  234396     A. Davies   18      Canada              73         88   
1  239085     E. Håland   18      Norway              79         90   
2  242816    Riqui Puig   19       Spain              71         87   
3  242641  R. Aït Nouri   18      France              70         85   
4  220883      R. Borré   23    Colombia              78         85   

                Team  Value Wage Value  Total Stats  
0  FC Bayern München    €8M       €21K         1935  
1  Borussia Dortmund   €20M       €32K         1906  
2       FC Barcelona    €5M       €39K         1793  
3         Angers SCO  €3.4M        €3K         1721  
4        River Plate   €15M       €25K         2037  


In [9]:
player_url = 'https://sofifa.com/player/'
detailed_columns = ["Full Name", "Birth Date", "Height", "Weight", "Position #1", "Position #2", 
                    "Preferred Foot", "International Reputation", "Weak Foot", "Skill Moves","Work Rate", "Body Type", "Real Face", "Release Clause",
                    "Current Team", "Jersey Number", "Joined","Contract Valid Until", 
                    "LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", 
                    "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB",                    
                    "Crossing", "Finishing", "Heading Accuracy", "Short Passing", "Volleys", 
                    "Dribbling", "Curve", "FK Accuracy", "Long Passing", "Ball Control", 
                    "Acceleration", "Sprint Speed", "Agility","Reactions", "Balance", 
                    "Shot Power", "Jumping", "Stamina", "Strength", "Long Shots", 
                    "Aggression", "Interceptions", "Positioning", "Vision",
                    "Penalties", "Composure", "Defensive Awareness", "Standing Tackle","Sliding Tackle", 
                    "GK Diving", "GK Handling", "GK Kicking","GK Positioning", "GK Reflexes", "ID"]
detailed_data = pd.DataFrame(index = range(0, data.count()[0]), columns = detailed_columns)
detailed_data.ID = data.ID
print(detailed_data)

      Full Name Birth Date Height Weight Position #1 Position #2  \
0           NaN        NaN    NaN    NaN         NaN         NaN   
1           NaN        NaN    NaN    NaN         NaN         NaN   
2           NaN        NaN    NaN    NaN         NaN         NaN   
3           NaN        NaN    NaN    NaN         NaN         NaN   
4           NaN        NaN    NaN    NaN         NaN         NaN   
5           NaN        NaN    NaN    NaN         NaN         NaN   
6           NaN        NaN    NaN    NaN         NaN         NaN   
7           NaN        NaN    NaN    NaN         NaN         NaN   
8           NaN        NaN    NaN    NaN         NaN         NaN   
9           NaN        NaN    NaN    NaN         NaN         NaN   
10          NaN        NaN    NaN    NaN         NaN         NaN   
11          NaN        NaN    NaN    NaN         NaN         NaN   
12          NaN        NaN    NaN    NaN         NaN         NaN   
13          NaN        NaN    NaN    NaN        

In [10]:
for id in detailed_data.ID:
    print(id)
    full_player_url = player_url + str(id)
    source_code = requests.get(full_player_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    skill_set = {}

    article = soup.find('article')
    meta_data = article.find('div', {'class': 'meta'})
    
    # Basic Data
    
    skill_set["Full Name"] = meta_data.text.split('  ')[0]
    
    # POSITIONS
    positions = meta_data.find_all('span', class_ = "pos")
    skill_set["Position #1"] = positions[0].text
    if(len(positions) > 1):
        skill_set["Position #2"] = positions[1].text
    else:
        skill_set["Position #2"] = None
    
    
    meta_data = meta_data.text.split(' ')
    length = len(meta_data)
    
    # BIRTH DATE
    year = meta_data[length- 3].split(')')[0]
    day = meta_data[length-4].split(',')[0]
    month = meta_data[length -5].split('(')[1]
    skill_set["Birth Date"] = months_correlatives[month] + '/'+ day + '/' + year
     
    # WEIGHT AND HEIGHT
    weight = meta_data[length - 1]
    height = meta_data[length - 2]
    skill_set["Weight"] = weight
    skill_set["Height"] = height
    
    
    
    teams = article.find('div', class_ = "teams")
    lis = teams.find('div', class_ = 'column col-6' ).find_all('li')
    for li in lis:
        label = li.find('label').text
        #print(label)
        value = li.text.replace(label, '').strip()
        skill_set[label] = value
    lis = teams.find_all('div', class_ = 'column col-5' )
    
    # Informations about the player at current team
    if(len(lis) == 0): #Player doesn't have current team
        if(len(lis[0].find_all('li')) == 0):
            lis = lis[1].find_all('li')
        else:
            lis = lis[0].find_all('li')


        skill_set["Current Team"] = lis[0].text

        for li in lis[3:]:
            label = li.find('label').text
            #print(label)
            value = li.text.replace(label, '').strip()
            if(label == "Joined"):
                value = value.split(' ')
                skill_set[label] = months_correlatives[value[0]] + '/' + value[1].split(',')[0] + '/' +value[2]
            else:
                skill_set[label] = value
    
    # Skill Set
    columns = article.find_all('div', class_ = "columns spacing")
    for column in columns:
        lis = column.find_all('li')
        for li in lis:
            value = re.findall(r'\d+', li.text)
            if(value == []):
                break
            value = int(value[0])
            #print(li.text)
            label = filter(None, re.split('[^a-zA-Z]*', li.text))
            label = ' '.join(label)
            #print(str(label))
            
            skill_set[str(label)] = value
            #print(str(label), skill_set[str(label)])
    
    if(skill_set["Position #1"] != "GK"):
        # ASIDE INFORMATION
        aside_information = soup.find('aside').find('div', class_ = "bp3-callout spacing calculated").find_all('div', class_ = "columns")
        for col in aside_information:
            divs = col.find_all('div', class_ = "column")
            for div in divs:
                if(div.find('div') != None):
                    label = div.find('div').text
                    #print(label)
                    value = div.text.replace(label, '').strip()
                    skill_set[label] = value
    
    for key, value in skill_set.items():
        detailed_data.loc[detailed_data.ID == id, key] = value
    
print(detailed_data.head(1))
detailed_data.to_csv('detailed_data', encoding = "utf-8")

234396
239085
242816
242641
220883
229880
232656
213956
189068
253004
212188
232104
244260
233049
20801
246104
251566
251854
241096
239053
225100
246147
248243
206517
245371
231281
209331
235790
231478
243812
213296
220018
212198
246045
243630
228702
242444
235212
203376
239956
234906
231677
244778
181291
222665
231747
192387
243414
221363
245388
238794
238067
179813
226627
239817
246960
252042
240060
41236
233306
229558
158023
235243
244840
237678
220697
201942
226271
237692
208722
237383
246669
204838
232432
251530
231521
223113
228092
226766
240740
246618
211300
235926
208670
241721
233064
183711
216267
243715
212831
228251
211110
212194
247394
240754
235735
252238
223848
220814
192816
236772
231352
246191
239231
238476
239890
237681
241568
237819
236007
245541
222331
246419
232293
236636
188545
223885
195864
230658
242602
228618
234378
246763
240507
209297
236403
172962
236920
229984
231447
233927
229906
231943
236632
226790
247888
234574
238160
231443
239970
239818
232756
226166
2

229773
244289
190915
205632
212811
194957
247229
240709
231969
207562
245287
222104
206516
228813
233795
177388
213991
228383
234128
222737
199767
220407
237841
223597
224158
231826
222160
246748
234924
228635
215455
238858
234468
240661
242287
203487
202695
236583
244859
219455
235161
243275
184252
181098
192638
171877
238370
240734
201155
220621
234730
226789
215502
200429
175789
233476
184432
233732
251105
229188
209960
226853
201519
199151
186832
240500
206511
231838
247827
194404
246155
200316
173221
229148
253072
221168
183569
230147
211269
220093
231089
189461
207537
224854
221600
221982
192227
241941
171897
159145
222352
241771
253444
219104
172522
225539
204399
244938
229636
241650
244622
210287
223033
254845
232223
120533
222077
232665
229585
239838
203362
216820
241727
229758
184431
219087
225945
202750
215725
214132
178562
225403
224921
216433
134744
236624
201143
240241
231247
223952
217870
224885
184484
221743
246145
221269
224309
45186
198657
207790
163587
224425
220714


177644
202851
206083
112316
165229
176930
206590
193698
175379
201884
146758
178091
180283
204550
191565
247412
234795
231041
243009
227106
224116
247509
230988
242332
234099
214876
245299
228738
241263
237153
243449
242201
229487
237960
244483
236393
214989
227854
236639
221354
241708
252463
252544
225028
214436
234930
219259
215927
215256
252290
213905
239747
223671
244952
204215
213829
189390
163155
213689
138110
210697
203843
207998
186146
212127
188155
180818
177134
202054
179847
173521
169588
199422
197170
211856
166844
210489
191687
184037
193116
207725
183556
199576
179591
171579
202445
204523
204289
158232
158144
212715
173426
157304
170597
204120
203725
136113
177633
246848
253061
247736
245035
243612
245283
243244
242052
244712
247246
246685
251948
246885
242118
245865
251809
208535
203668
234525
241858
212204
239397
211108
193886
239097
206134
239763
190972
206006
213432
241036
239580
206086
227788
221671
195479
202849
193133
224389
216468
220197
206538
210635
223938
215119

241861
241379
225863
225701
239950
239755
240351
238097
232278
240453
234785
240285
235916
235036
241736
235046
235623
210450
204652
201938
213577
225310
215086
200778
220540
202078
215817
213948
210951
224489
223603
213814
204346
214446
210441
222587
224855
221695
210320
212230
222467
225252
201982
219523
220815
212192
216811
201368
211818
201215
205850
221279
201041
210556
220638
213407
213536
224423
204506
212783
201533
222709
205086
208719
202024
211176
204277
208251
218534
207952
199158
205756
200309
202491
204193
224869
212476
152747
193177
173818
176619
196688
194017
189451
192587
190461
171875
192110
198513
192613
185137
191055
192575
177159
191390
1238
189680
196147
190502
135451
188005
190752
158133
50542
189446
177922
172175
197231
191628
163631
198683
137677
188274
184467
184626
172862
192397
178415
178255
191151
186422
163806
169317
198261
190761
175895
193469
167943
189795
254067
252932
254268
253025
254480
252943
254502
247833
247014
244403
252052
243001
243468
251147
24

221358
224130
224541
216227
211068
218154
215051
212350
221331
220019
215247
216289
211221
212449
212943
210236
220075
225117
223212
211119
211706
216275
222840
223962
220172
216814
220139
223344
219496
221662
214129
211348
224129
214701
221641
221737
222515
223682
192930
208302
201976
202107
200773
207899
201908
208543
200841
202123
194806
199729
204226
203067
198617
205753
201061
192841
193267
200113
192594
198379
200302
193425
196935
208047
194018
204125
209675
202061
209937
204464
203746
206306
206524
198198
200519
204247
203884
203980
199833
208153
206562
204992
197031
193062
209621
205478
201144
194964
201073
207956
192722
203570
203736
196150
203280
201403
186783
110638
135742
186578
118794
169426
182629
189472
122849
183284
192001
190799
179875
169600
177294
188289
165517
179574
188955
184480
168312
192350
158851
178416
184624
168621
2702
182883
190868
170320
184982
180334
190745
155984
190724
174549
177509
189403
173735
183871
183982
167452
187688
172233
158777
186148
190491
2

...

232540
232577
232507
232490
232332
232543
232465
232511
232479
232505
232372
232574
232596
232426
232422
232500
232512
232127
232027
232118
231718
231722
232054
231877
232049
231711
232196
231674
232065
231937
231704
231572
231563
232136
231916
231553
231828
231912
231929
231829
231867
231664
232071
232078
231649
231945
232002
232083
231632
232117
231956
231706
231842
231844
232013
232055
232142
232007
231661
231917
231834
232095
231646
232081
231855
232030
232050
232101
231947
232110
231682
231556
232130
231993
231681
231911
231123
231455
231105
231079
231458
231103
231219
231273
231279
231403
231018
231500
231137
231226
231498
231096
231119
231278
231222
230969
231185
231052
231269
231199
231186
231290
231139
230980
231502
231386
231464
231033
230974
231038
231319
231246
231303
231479
231125
231065
230981
231525
231009
231233
231354
231223
231171
231539
231116
230971
231237
231258
231031
231056
230962
230923
231069
231484
231300
230713
230750
230392
230801
230403
230400
230572
230785

217826
217647
217972
218543
218424
219095
217810
217711
217589
219122
219172
217994
218363
219449
217610
218047
218651
217817
218656
218591
217803
218938
219167
218170
219074
219473
218576
218657
216666
216472
217140
216610
216407
217047
216486
216867
216689
216374
216522
216501
216503
216647
216580
216794
216728
216442
217148
217217
216351
216763
216690
216789
217332
217201
216508
216799
217564
216498
217225
216833
216403
216389
217118
216671
217202
216805
217138
217007
216583
216568
216623
217391
216741
216327
216703
216688
217191
216773
216385
217077
216979
216736
216428
216823
216597
217082
217030
216313
215434
216021
215398
215573
215974
215676
216211
215495
216113
215389
215379
215571
215604
215808
216283
215492
216301
216055
216005
216078
215517
216025
215540
215380
215487
216303
216097
216190
215686
216174
215738
215553
216300
216116
216134
215695
216319
215791
216020
215609
216233
215401
216314
215582
215949
215942
215436
215443
215668
215737
216117
216304
215373
216269
216184

202921
202888
202459
202975
202570
202986
202456
202904
202430
202949
202420
202881
202766
202930
202802
202783
202692
202708
202561
202896
202673
202533
202610
202381
202552
202748
202659
202749
202703
202660
202832
202786
202810
202932
202526
202973
202892
202395
202988
202559
202915
202148
202343
201883
202155
202342
202049
202172
202143
202072
201944
202312
201856
201880
202202
201846
201941
202084
202040
202149
202338
201498
201983
202207
202000
201890
202162
201873
202215
202025
202069
201482
202223
201709
201949
202180
201576
201529
202105
201845
202082
201921
202322
201534
201814
201978
202285
202182
201527
202146
202243
202022
201522
201853
202050
201888
202279
202077
201924
201917
201940
200942
201212
201149
201352
201094
200911
201467
200818
201072
200953
200963
201460
201120
201452
201409
201327
200968
201029
200931
200983
201238
201105
200854
201082
200936
201217
201450
201146
201353
200798
200823
201374
200887
200834
200821
200913
200930
201080
201411
201019
201295
201038

181017
179768
179767
179831
181247
181044
181022
181021
180284
181377
180929
180840
179869
181106
179822
181223
179855
180945
180400
179917
181014
180717
180229
180826
181407
180405
180716
180800
179871
181375
181181
180726
179840
181269
180191
180740
180808
181175
181194
181307
180887
179911
179872
179941
179983
180749
180433
180627
180838
180796
178180
178452
178264
178190
178233
178220
179613
178247
178485
178766
179071
178565
178187
178421
178158
179731
178623
178481
179668
178167
178121
178768
178425
178316
178177
178461
178139
179646
178199
178363
178232
178168
178414
179616
178215
178207
178317
179703
178221
178746
179708
178098
178430
178773
178281
179569
178715
178297
179736
178283
179563
178589
178585
179515
179509
178246
178590
179663
178319
178223
177974
178087
177698
177904
178054
177668
178042
177931
177699
177860
178022
177495
177639
177953
177558
177915
177672
177549
177473
177516
177801
177590
177899
178033
177493
177550
177875
178008
178006
177522
177584
177656
177877

141100
139169
139443
140164
138839
136342
137823
138810
137460
138698
138874
138703
137686
138297
138336
137693
136631
137553
138630
137179
138782
138427
138468
138456
138787
138989
137440
138984
139016
138670
138872
136363
136365
138729
138457
138086
139017
138918
138066
139010
137208
137770
137699
137429
138413
137380
138181
137519
138334
138715
137350
138585
138473
137166
137671
138986
138251
138817
138463
138250
138809
136601
138249
137074
117896
136161
122057
135016
117566
121622
134946
135084
134955
123581
121185
135527
118261
135423
135218
134621
117364
134792
135819
135465
136137
133249
134783
125167
135475
130933
135537
135515
125397
123603
136248
118704
134734
134627
135435
134828
135098
119422
122649
136070
135866
123629
123586
135190
122417
119655
117062
135524
120110
122718
122066
121721
134812
124949
117402
135804
120280
123621
135401
136222
113348
110366
113710
112554
110652
105530
109416
114123
112778
112373
112251
115519
116024
104476
107261
116494
110657
101891
111008

In [13]:
data_csv = pd.read_csv('detailed_data')
data = pd.DataFrame(data_csv, columns = detailed_columns)
print(data)

                                 Full Name  Birth Date Height  Weight  \
0                          Alphonso Davies   11/2/2000  5'11"  159lbs   
1                      Erling Braut Håland   7/21/2000   6'4"  192lbs   
2                        Ricard Puig Martí   8/13/1999   5'7"  123lbs   
3                          Rayan Aït Nouri    6/6/2001  5'10"  154lbs   
4                      Rafael Santos Borré   9/15/1995   5'9"  157lbs   
5                        Aaron Wan-Bissaka  11/26/1997   6'0"  159lbs   
6          Theo Bernard François Hernández   10/6/1997   6'0"  181lbs   
7                      Adama Traoré Diarra   1/25/1996  5'10"  159lbs   
8                                 Bas Dost   5/31/1989   6'5"  196lbs   
9                           Anssumane Fati  10/31/2002  5'10"  146lbs   
10                             Timo Werner    3/6/1996  5'11"  168lbs   
11                            Daniel James  11/10/1997   5'7"  168lbs   
12                    Daniel Olmo Carvajal    5/7/1